In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from sklearn.metrics import f1_score

import tensorflow as tf

In [2]:
# Helper functions

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

def normalize_data(data):
    # Data pre-processing, Normalize each image with itself
    n = data.shape[0]
    for i in range(n):
        xx = data[i,:,:]
        xx -= np.mean(xx) # Centering in 0
        xx /= np.linalg.norm(xx) # Normalizing to 1
        data[i] = xx # Affect value
    return data
def one_hot_convert(vector, num_classes=None):
    """ (From https://stackoverflow.com/questions/29831489/numpy-1-hot-array)
    Converts an input 1-D vector of integers into an output
    2-D array of one-hot vectors, where an i'th input value
    of j will set a '1' in the i'th row, j'th column of the
    output array.

    Example:
        v = np.array((1, 0, 4))
        one_hot_v = convertToOneHot(v)
        print one_hot_v

        [[0 1 0 0 0]
         [1 0 0 0 0]
         [0 0 0 0 1]]
    """

    assert isinstance(vector, np.ndarray)
    assert len(vector) > 0

    if num_classes is None:
        num_classes = np.max(vector)+1
    else:
        assert num_classes > 0
        assert num_classes >= np.max(vector)

    result = np.zeros(shape=(len(vector), num_classes))
    result[np.arange(len(vector)), vector] = 1
    return result

In [3]:
# Loaded a set of images
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
#n = min(20, len(files)) # Load maximum 20 images
n = len(files)
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

n = int(len(files)*0.5) 
#n=10 # Only use 10 images for training
np_imgs = normalize_data(np.asarray(imgs))
train_imgs = np_imgs[:n]
val_imgs = np_imgs[n:]
np_gt = np.ceil(np.asarray(gt_imgs))
train_gt = np_gt[:n]
val_gt = np_gt[n:]
train_size = train_gt.shape[0]
val_size = val_gt.shape[0]
print(train_size)
print(val_size)

Loading 100 images
satImage_001.png
Loading 100 images
satImage_001.png
50
50


In [4]:
def weighted_pixelwise_crossentropy(y_true, y_pred, class_weights):
    """computes pixelwise crossentropy"""
    temp = tf.clip_by_value(y_pred, 1e-8, 1. - 1e-8)
    loss = - tf.reduce_mean(tf.multiply(y_true * tf.log(temp), class_weights))
    return loss

In [ ]:
learning_rate = 1e-2
reg = 1e-3
n_filters = 32
kernel_size = 3

tf_data = tf.placeholder(tf.float32,[None, 400,400,3])
tf_labels = tf.placeholder(tf.int32,[None,2])
keep_prob = tf.placeholder(tf.float32)
regularizer = tf.contrib.layers.l2_regularizer(scale=reg)

class_weights = tf.constant([[1.0,10.0]])
weights = tf.reduce_sum(class_weights * tf.cast(tf_labels, tf.float32), axis=1)

conv1 = tf.layers.conv2d(inputs=tf_data, filters=n_filters, kernel_size=kernel_size, kernel_regularizer=regularizer, activation=tf.nn.relu, padding='SAME')
print("conv1 size", conv1.shape)

pool1 = tf.contrib.layers.max_pool2d(inputs=conv1, kernel_size=2, stride=2)
print("pool1 size", pool1.shape)

conv2 = tf.layers.conv2d(inputs=pool1, filters=n_filters, kernel_size=kernel_size, kernel_regularizer=regularizer, activation=tf.nn.relu, padding='SAME')
print("conv2 size", conv2.shape)

pool2 = tf.contrib.layers.max_pool2d(inputs=conv2, kernel_size=2, stride=2)
print("pool2 size", pool2.shape)

deconv1 = tf.layers.conv2d_transpose(inputs=pool2, filters=n_filters, kernel_size=4, strides=2, kernel_regularizer=regularizer, activation=tf.nn.relu, padding='SAME')
print("deconv1 size", deconv1.shape)

deconv2 = tf.layers.conv2d_transpose(inputs=deconv1, filters=n_filters, kernel_size=4, strides=2, kernel_regularizer=regularizer, activation=tf.nn.relu, padding='SAME')
print("deconv2 size", deconv2.shape)

score_layer = tf.layers.conv2d(inputs=deconv2, filters=2, kernel_size=1,kernel_regularizer=regularizer)
print("score size", score_layer.shape)

logits = tf.reshape(score_layer, (-1,2))

#trn_labels=tf.reshape(tf_labels, [-1])

#cross_entropy=tf.nn.softmax_cross_entropy_with_logits(logits,trn_labels,name='x_ent')
#loss=tf.reduce_mean(cross_entropy, name='x_ent_mean')
#train_op=tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(loss,global_step=global_step)

#cross_entropy = weighted_pixelwise_crossentropy(tf.cast(tf_labels, dtype=tf.float32), logits, class_weights)
cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels=tf_labels, logits=logits, weights=weights)
reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term = tf.contrib.layers.apply_regularization(regularizer, reg_variables)
loss = reg_term + cross_entropy

train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

preds = tf.argmax(logits,axis=1,output_type=tf.int32)
print(logits.shape)
preds= tf.argmax(tf.reshape(tf.nn.softmax(logits), (-1,400,400,2)), axis=3)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.cast(tf.reshape(preds, [-1]), tf.int32), tf.argmax(tf_labels,axis=1, output_type=tf.int32)), tf.float32))

conv1 size (?, 400, 400, 32)
pool1 size (?, 200, 200, 32)
conv2 size (?, 200, 200, 32)
pool2 size (?, 100, 100, 32)
deconv1 size (?, 200, 200, 32)
deconv2 size (?, 400, 400, 32)
score size (?, 400, 400, 2)
(?, 2)


In [ ]:
##Model inference

n_epoch = 10

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(n_epoch):
    train_idx = np.random.permutation(np.arange(n))
    for idx in train_idx:
        print("training on image {:d}".format(idx))
        cur_img = train_imgs[idx]

        batch_x = np.expand_dims(cur_img,axis=0)
        batch_y = one_hot_convert(np.reshape(train_gt[idx],(400*400)).astype(int),2)
        #batch_y = np.reshape(train_gt[idx],(400*400)).astype(int)
        

        _, train_acc, train_loss, train_cross_entropy, train_reg_term = sess.run([train_step, accuracy, loss, cross_entropy, reg_term], feed_dict={tf_data : batch_x, tf_labels : batch_y})
        print(train_cross_entropy)
        print(train_reg_term)
    if epoch % 1 ==0:
        train_pred, train_acc, train_loss = sess.run([preds, accuracy, loss], feed_dict={tf_data : train_imgs, tf_labels : one_hot_convert(np.reshape(train_gt,train_size*400*400).astype(int),2)})
        val_pred, val_acc, val_loss = sess.run([preds, accuracy, loss], feed_dict={tf_data : val_imgs, tf_labels : one_hot_convert(np.reshape(val_gt,val_size*400*400).astype(int),2)})
        
        print(train_pred.shape)
        print(val_pred.shape)
        print(train_gt.shape)
        f1_train = f1_score(np.reshape(train_gt,train_size*400*400), np.reshape(train_pred, [-1]), average='macro') 
        f1_val = f1_score(np.reshape(val_gt,val_size*400*400), np.reshape(val_pred,[-1]), average='macro')
        print("epoch ", epoch+1,", val f1 : ", f1_val, ", train f1 : ", f1_train)
        print("val_loss : ", val_loss, ", train_loss : ", train_loss)
        


training on image 18
1.43851
3.94809e-07
training on image 33
2.15495
4.00982e-07
training on image 46
1.76863
4.19109e-07
training on image 2
1.93016
4.56878e-07
training on image 0
1.72375
4.45525e-07
training on image 9
1.84936
4.39952e-07
training on image 5
1.96785
4.43413e-07
training on image 26
1.50086
4.52692e-07
training on image 36
1.91742
4.61032e-07
training on image 29
1.60662
4.73079e-07
training on image 27
1.35572
4.83268e-07
training on image 31
1.58254
4.86131e-07
training on image 48
1.64476
4.89006e-07
training on image 6
1.88996
4.92562e-07
training on image 15
1.55168
4.97759e-07
training on image 8
1.9742
5.02758e-07
training on image 41
1.92575
5.0861e-07
training on image 17
1.55635
5.14879e-07
training on image 3
1.7129
5.20808e-07
training on image 30
1.96411
5.26132e-07
training on image 25
1.8996
5.32097e-07
training on image 24
1.78139
5.39172e-07
training on image 16
1.66392
5.46764e-07
training on image 10
1.62334
5.51945e-07
training on image 38
1.743


In [ ]:
plt.figure(figsize=(16, 5))
plt.subplot(1,3,1)
plt.imshow(val_gt[1,:])
plt.subplot(1,3,2)
plt.imshow(val_gt[2,:])
plt.subplot(1,3,3)
plt.imshow(val_gt[3,:])

In [ ]:
test = np.reshape(val_pred, val_gt.shape)
plt.figure(figsize=(16, 5))
plt.subplot(1,3,1)
plt.imshow(test[1,:])
plt.subplot(1,3,2)
plt.imshow(test[2,:])
plt.subplot(1,3,3)
plt.imshow(test[3,:])

In [ ]:
conv_1, pool_1, conv_2, pool_2, deconv_1, deconv_2, score= sess.run([conv1, pool1, conv2, pool2, deconv1, deconv2, score_layer], feed_dict={tf_data : np.expand_dims(train_imgs[1], axis=0), tf_labels : one_hot_convert(np.reshape(train_gt[1],400*400).astype(int),2)})

print(max(np.reshape(conv_1, [-1])))
print(max(np.reshape(pool_1, [-1])))
print(max(np.reshape(conv_2, [-1])))
print(max(np.reshape(pool_2, [-1])))
print(max(np.reshape(deconv_1, [-1])))
print(max(np.reshape(deconv_2, [-1])))
print(max(np.reshape(score, [-1])))

plt.figure(figsize=(16,8))
plt.subplot(2,4,1)
plt.imshow((np.asarray(np.squeeze(conv_1)[:,:,1])))
plt.subplot(2,4,2)
plt.imshow((np.asarray(np.squeeze(pool_1)[:,:,1])))
plt.subplot(2,4,3)
plt.imshow((np.asarray(np.squeeze(conv_2)[:,:,1])))
plt.subplot(2,4,4)
plt.imshow((np.asarray(np.squeeze(pool_2)[:,:,1])))
plt.subplot(2,4,5)
plt.imshow((np.asarray(np.squeeze(deconv_1)[:,:,1])))
plt.subplot(2,4,6)
plt.imshow((np.asarray(np.squeeze(deconv_2)[:,:,1])))
plt.subplot(2,4,7)
plt.imshow((np.asarray(np.squeeze(score)[:,:,0])))
